In [26]:
import numpy as np
import pandas as pd
import os
import warnings
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, merge, Dropout, BatchNormalization
from keras.models import Model
from tensorflow.keras.utils import plot_model
from IPython.display import SVG
warnings.filterwarnings('ignore')
%matplotlib inline

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
csvPath = "/content/drive/MyDrive/RecSys/dataWithRatings.csv"
df = pd.read_csv(csvPath)
df1 = df[['user_id', 'name', 'Rating M1']].copy()

df1.rename(columns = {df1.columns[2]:'rating1'}, inplace=True)
df1['game_id'] = df.groupby(['name']).ngroup()
df1['gamer_id'] = df.groupby(['user_id']).ngroup()
df1_toTrain = df1.copy().drop(columns={'name','user_id'})
df1_toTrain.head()

,rating1,game_id,gamer_id
0,5,469,3878
1,5,469,1652
2,1,469,337
3,3,469,6663
4,1,469,2534


In [29]:
train, test = train_test_split(df1_toTrain, test_size=0.85, random_state=0)

In [30]:
print(df1_toTrain.shape)

(36257, 3)


In [31]:
n_items = int(df1_toTrain['game_id'].max())
n_users = int(df1_toTrain['gamer_id'].max())
print(n_items)
print(n_users)

1514
9253


In [38]:
latent_dim = 50
# Define inputs
game_input = Input(shape=[1],name='game-input')
user_input = Input(shape=[1], name='user-input')
# MLP Embeddings
game_embedding_mlp = Embedding(n_items + 1, latent_dim, name='game-embedding-mlp')(game_input)
game_vec_mlp = Flatten(name='flatten-game-mlp')(game_embedding_mlp)
user_embedding_mlp = Embedding(n_users + 1, latent_dim, name='user-embedding-mlp')(user_input)
user_vec_mlp = Flatten(name='flatten-user-mlp')(user_embedding_mlp)
# MF Embeddings
game_embedding_mf = Embedding(n_items + 1, latent_dim, name='game-embedding-mf')(game_input)
game_vec_mf = Flatten(name='flatten-game-mf')(game_embedding_mf)
user_embedding_mf = Embedding(n_users + 1, latent_dim, name='user-embedding-mf')(user_input)
user_vec_mf = Flatten(name='flatten-user-mf')(user_embedding_mf)

concat = merge.concatenate([game_vec_mlp, user_vec_mlp], name='concat')
concat_dropout = Dropout(0.2)(concat)

fc_1 = Dense(100, name='fc-1', activation='relu')(concat_dropout)
fc_1_bn = BatchNormalization(name='batch-norm-1')(fc_1)
fc_1_dropout = Dropout(0.2)(fc_1_bn)
fc_2 = Dense(50, name='fc-2', activation='relu')(fc_1_dropout)
fc_2_bn = BatchNormalization(name='batch-norm-2')(fc_2)
fc_2_dropout = Dropout(0.2)(fc_2_bn)
# Prediction from both layers
pred_mlp = Dense(10, name='pred-mlp', activation='relu')(fc_2_dropout)
pred_mf = merge.dot(inputs=[game_vec_mf, user_vec_mf], axes = 1, name='pred-mf')
combine_mlp_mf = merge.concatenate([pred_mf, pred_mlp], name='combine-mlp-mf')
# Final prediction
result = Dense(1, name='result', activation='relu')(combine_mlp_mf)
model = Model([user_input, game_input], result)
model.compile('adam', 'mean_squared_error')


In [39]:
history = model.fit([train.gamer_id, train.game_id], train.rating1, epochs=100, verbose=1)

Epoch 1/100
170/170 [==============================] - 3s 8ms/step - loss: 5.0278
Epoch 2/100
170/170 [==============================] - 1s 8ms/step - loss: 2.6641
Epoch 3/100
170/170 [==============================] - 1s 8ms/step - loss: 2.0676
Epoch 4/100
170/170 [==============================] - 1s 8ms/step - loss: 1.4813
Epoch 5/100
170/170 [==============================] - 1s 8ms/step - loss: 0.9917
Epoch 6/100
170/170 [==============================] - 1s 8ms/step - loss: 0.6873
Epoch 7/100
170/170 [==============================] - 1s 8ms/step - loss: 0.4794
Epoch 8/100
170/170 [==============================] - 1s 8ms/step - loss: 0.3856
Epoch 9/100
170/170 [==============================] - 1s 9ms/step - loss: 0.3103
Epoch 10/100
170/170 [==============================] - 1s 8ms/step - loss: 0.2751
Epoch 11/100
170/170 [==============================] - 1s 8ms/step - loss: 0.2436
Epoch 12/100
170/170 [==============================] - 1s 9ms/step - loss: 0.2051
Epoch 13/100


In [40]:
predictions = model2.predict([test.gamer_id.head(10), test.game_id.head(10)])
[print(predictions[i], test.rating1.iloc[i]) for i in range(0,10)]

[1.7058425] 5
[2.8365474] 4
[2.6905427] 5
[2.9112248] 5
[1.060725] 1
[0.756301] 1
[4.026786] 1
[1.0482664] 5
[2.0014212] 5
[1.9707394] 1


[None, None, None, None, None, None, None, None, None, None]

In [41]:
results = model2.evaluate([test.gamer_id, test.game_id], test.rating1)

964/964 [==============================] - 2s 3ms/step - loss: 4.4467
